In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
from keras.callbacks import EarlyStopping, TensorBoard
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
import plotly.graph_objects as go



# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

/kaggle/input/for-simple-exercises-time-series-forecasting/Miles_Traveled.csv
/kaggle/input/for-simple-exercises-time-series-forecasting/Alcohol_Sales.csv
/kaggle/input/for-simple-exercises-time-series-forecasting/BeerWineLiquor.csv
/kaggle/input/for-simple-exercises-time-series-forecasting/energydata_complete.csv
/kaggle/input/for-simple-exercises-time-series-forecasting/prophet_facebook2.png
/kaggle/input/for-simple-exercises-time-series-forecasting/Deep Learning for Time Series Forecasting.png


# Load dataset


In [2]:
# Load dataset
data = pd.read_csv("/kaggle/input/for-simple-exercises-time-series-forecasting/Miles_Traveled.csv")
dataset = data['TRFVOLUSM227NFWA'].values
dataset = dataset.astype('float32')

In [3]:
# Normalize dataset
scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(dataset.reshape(-1, 1))

# Split into train and test sets


In [4]:
# Split into train and test sets
train_size = int(len(dataset) * 0.67)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]

# Convert an array of values into a dataset matrix


In [5]:
# Convert an array of values into a dataset matrix
def create_dataset(dataset, look_back=3):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), 0]
        dataX.append(a)
        dataY.append(dataset[i + look_back, 0])
    return np.array(dataX), np.array(dataY)

In [6]:
# Create datasets for ML
look_back = 1
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)

In [7]:
# Reshape into X=t and Y=t+1
look_back = 3
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)

In [8]:
# Reshape input to be [samples, time steps, features]
trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

# Create and fit the LSTM network


In [9]:
# Create and fit the LSTM network
model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(1, look_back)))
model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dropout(0.2))
model.add(Dense(1))

model.compile(loss='mean_squared_error', optimizer='adam')

# Define EarlyStopping


In [10]:
# Define EarlyStopping
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)

model.fit(trainX, trainY, validation_data=(testX, testY), epochs=50, batch_size=64, verbose=2, callbacks=[es])

Epoch 1/50
7/7 - 9s - loss: 0.1809 - val_loss: 0.5732 - 9s/epoch - 1s/step
Epoch 2/50
7/7 - 0s - loss: 0.1418 - val_loss: 0.4542 - 81ms/epoch - 12ms/step
Epoch 3/50
7/7 - 0s - loss: 0.1009 - val_loss: 0.3160 - 67ms/epoch - 10ms/step
Epoch 4/50
7/7 - 0s - loss: 0.0586 - val_loss: 0.1721 - 67ms/epoch - 10ms/step
Epoch 5/50
7/7 - 0s - loss: 0.0262 - val_loss: 0.0595 - 71ms/epoch - 10ms/step
Epoch 6/50
7/7 - 0s - loss: 0.0151 - val_loss: 0.0135 - 67ms/epoch - 10ms/step
Epoch 7/50
7/7 - 0s - loss: 0.0168 - val_loss: 0.0104 - 68ms/epoch - 10ms/step
Epoch 8/50
7/7 - 0s - loss: 0.0156 - val_loss: 0.0221 - 79ms/epoch - 11ms/step
Epoch 9/50
7/7 - 0s - loss: 0.0121 - val_loss: 0.0332 - 67ms/epoch - 10ms/step
Epoch 10/50
7/7 - 0s - loss: 0.0114 - val_loss: 0.0342 - 79ms/epoch - 11ms/step
Epoch 11/50
7/7 - 0s - loss: 0.0107 - val_loss: 0.0251 - 67ms/epoch - 10ms/step
Epoch 12/50
7/7 - 0s - loss: 0.0096 - val_loss: 0.0146 - 67ms/epoch - 10ms/step
Epoch 13/50
7/7 - 0s - loss: 0.0087 - val_loss: 0.008

# Make predictions


In [11]:
# Make predictions
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)

6/6 [==============================] - 0s 3ms/step


# Invert predictions


In [12]:
# Invert predictions
trainPredict = scaler.inverse_transform(trainPredict)
trainY = scaler.inverse_transform([trainY])
testPredict = scaler.inverse_transform(testPredict)
testY = scaler.inverse_transform([testY])

# Evaluate Performance


In [13]:
# Evaluate Performance
trainScore = mean_squared_error(trainY, np.reshape(trainPredict, trainY.shape))
print('Train Score: %.2f MSE' % (trainScore))

testScore = mean_squared_error(testY, np.reshape(testPredict, testY.shape))
print('Test Score: %.2f MSE' % (testScore))

trainScore = np.sqrt(mean_squared_error(trainY, np.reshape(trainPredict, trainY.shape)))
print('Train Score: %.2f RMSE' % (trainScore))

testScore = np.sqrt(mean_squared_error(testY, np.reshape(testPredict, testY.shape)))
print('Test Score: %.2f RMSE' % (testScore))

trainScore = mean_absolute_error(trainY, np.reshape(trainPredict, trainY.shape))
print('Train Score: %.2f MAE' % (trainScore))

testScore = mean_absolute_error(testY, np.reshape(testPredict, testY.shape))
print('Test Score: %.2f MAE' % (testScore))

Train Score: 151987155.82 MSE
Test Score: 324873548.67 MSE
Train Score: 12328.31 RMSE
Test Score: 18024.25 RMSE
Train Score: 10653.19 MAE
Test Score: 15043.02 MAE


# Visualization

In [14]:
# Create a dataset same size as our original dataset filled with NaNs
trainPredictPlot = np.empty_like(dataset)
trainPredictPlot[:, :] = np.nan

# Replace the positions with the training predictions in the time they belong
trainPredictPlot[look_back:len(trainPredict)+look_back, :] = trainPredict

# Create a dataset same size as our original dataset filled with NaNs
testPredictPlot = np.empty_like(dataset)
testPredictPlot[:, :] = np.nan

# Replace the positions with the testing predictions in the time they belong
testPredictPlot[len(trainPredict)+(look_back*2)+1:len(dataset)-1, :] = testPredict

# Create traces
fig = go.Figure()

# Original data trace
fig.add_trace(go.Scatter(y=scaler.inverse_transform(dataset).flatten(), mode='lines', name='original data'))

# Train data trace
fig.add_trace(go.Scatter(y=trainPredictPlot.flatten(), mode='lines', name='train prediction'))

# Test data trace
fig.add_trace(go.Scatter(y=testPredictPlot.flatten(), mode='lines', name='test prediction'))

# Show plot
fig.show()


## Residual plot for the training data


In [15]:
# Residual plot for the training data
train_residuals = trainY[0] - trainPredict[:,0]

fig = go.Figure()
fig.add_trace(go.Scatter(y=train_residuals, mode='markers', name='Train residuals'))
fig.update_layout(title='Residuals of training data', xaxis_title='Index', yaxis_title='Residual')
fig.show()

# Residual plot for the testing data
test_residuals = testY[0] - testPredict[:,0]

fig = go.Figure()
fig.add_trace(go.Scatter(y=test_residuals, mode='markers', name='Test residuals'))
fig.update_layout(title='Residuals of testing data', xaxis_title='Index', yaxis_title='Residual')
fig.show()

## Scatter plot for training data


In [16]:
# Scatter plot for training data
fig = go.Figure()
fig.add_trace(go.Scatter(x=trainY[0], y=trainPredict[:,0], mode='markers', name='Train'))
fig.update_layout(title='Training data: Actual vs Predicted', xaxis_title='Actual', yaxis_title='Predicted', autosize=False, width=500, height=500,)
fig.add_shape(type="line", x0=min(trainY[0]), y0=min(trainPredict[:,0]), x1=max(trainY[0]), y1=max(trainPredict[:,0]), line=dict(color="MediumPurple",width=2))
fig.show()

# Scatter plot for testing data
fig = go.Figure()
fig.add_trace(go.Scatter(x=testY[0], y=testPredict[:,0], mode='markers', name='Test'))
fig.update_layout(title='Testing data: Actual vs Predicted', xaxis_title='Actual', yaxis_title='Predicted', autosize=False, width=500, height=500,)
fig.add_shape(type="line", x0=min(testY[0]), y0=min(testPredict[:,0]), x1=max(testY[0]), y1=max(testPredict[:,0]), line=dict(color="MediumPurple",width=2))
fig.show()


## 3D scatter plot of time, actual and predicted


In [17]:
# 3D scatter plot of time, actual and predicted
time = list(range(len(testY[0])))
fig = go.Figure(data=[go.Scatter3d(x=time, y=testY[0], z=testPredict[:,0], mode='markers')])
fig.update_layout(title='Time vs Actual vs Predicted', 
                  scene=dict(xaxis_title='Time',
                             yaxis_title='Actual values',
                             zaxis_title='Predicted values'))
fig.show()